# Test breakdown generator

In [1]:
PROMPT_JSON_EXTRACTION = """
# Prompt: Transform Legal Text into structured JSON representation

## Objective

Convert the provided legal text into structured JSON representation structure following the step by step below.

## Step by Step

1. Break the paragraph down into **individual sentences** and convert them to the **singular tense**.
    - Every subject, verb and complement must be converted into *singular tense**.

2. For each sentence do the following relevant information:
    - Subject: Entity (vehicle, person, etc.) that performs the action described in the conclusion.
    - Conclusion: The action that is present in the main sentence.
    - Deontic Modality: Optional. The importance of the action indicated in the conclusion. Either, should/should not or must/must not, can/can not.
    - Conditions: Things that have to be true, in order to the conclusion to be applicable.
    - Syntactic structure: Sub-divide both conclusion and conditions in: subject, deontic modality (for conclusion only), action (verb), and complement (place, object, etc.). If conclusion or conditions are **None**, syntactic structure must be empty.

3. Present the JSON with the extracted elements for each sentence.

The JSON must have the following structure.
- Every sentence will become a JSON object.
- Every subject and verb and complement must be converted into *singular tense**
- Each JSON object will have the "source_text" and the "syntactic_structure".
- The "source_text" is the original sentence.
- "syntactic_structure" contains the structure syntatic information for both conclusions and conditions, if any. That is, "subject", "deontic", "verb" and "complement".
- If the subject is "you" replace by "vehicle".
- If the verb relates to intent, such as "wants", "will", "intends", replace it with "intends".
- If there is no explicit deontic operator, assume "can" if applicable.
- Convert everything to **singular tense**.

Example:
```json
[
    {
        "source_text": "[Original sentence]",
        "syntactic_structure": {
            "conclusion": {
                "subject": "[Subject, if any, or null]",
                "deontic": "[Deontic operator, if any, or null]",
                "verb": "[Verb, if any, or null]",
                "object": "[Object, if any, or null]",
            },
            "conditions": [
                {
                    "subject": "[Subject, if any, or null]",
                    "verb": "[Verb, if any, or null]",
                    "object": "[Object, if any, or null]",
                }
                [More conditions if any...]
            ]
        }
    },
]
```


## How to find conclusions and conditions

### Conclusions

Follow bellow some details on how to extract the conclusions.

- *Predicate name* is the deontic modality if it exists (must, should, etc.). If not assume "can".
- Arguments are the subject, action and complements (if they exist). A complement can be a location, a time, a characteristic.

### Conditions

Follow bellow some details on how to extract the conditions.

- Find the subject, the verb, and the complement.
- Use the verb as the predicate name.
- The arguments are the subject and complements.
- USE **singular** to write subject, verb, and complement.


## Few Shot examples

### Example 1:

**Text:** You MUST stop behind the line at a junction with a 'Stop' sign and a solid white line across the road. Wait for a safe gap in the traffic before you move off.

**## Break the paragraph down into **individual sentences** and convert them to the singular tense**

1. You MUST stop behind the line at a junction with a 'Stop' sign and a solid white line across the road.
2. Wait for a safe gap in the traffic before you move off.

**Step 2: For each phrase, extract the relevant information**

1. You MUST stop behind the line at a junction with a 'Stop' sign and a solid white line across the road.

- Subject: "vehicle"
- Conclusion: "stop behind the line at a junction"
- Deontic modality: "must"
- Conditions:
    * Singular: "the junction has a 'Stop' sign"
    * Singular: "the junction has a solid white line across the road"
- Syntactic structure (Singular tense):
    Conclusion:
        * Subject: "vehicle", Deontic: "must", Verb: "stop", Complement: "behind the line at a junction"
    Conditions:
        * Subject: "junction", Verb: "has", Complement: "a 'Stop' sign"
        * Subject: "junction", Verb: "has", Complement: "a solid white line across the road"


2. Wait for a safe gap in the traffic before you move off.

- Subject: "vehicle"
- Conclusion: "move off"
- Deontic modality: None
- Conditions:
    * Singular: "there is a safe gap in the traffic"

- Syntactic structure (Singular tense):
    Conclusion:
        * Subject: "vehicle", Deontic: "can", Verb: "move off", Complement: "behind the line at a junction"
    Conditions:
        * Subject: "a safe gap", Verb: "is", Complement: "in the traffic"


**Step 3: Present the JSON**

```json
[
    {
        "source_text": "You MUST stop behind the line at a junction with a 'Stop' sign and a solid white line across the road.",
        "syntactic_structure": {
            "conclusion": {
                "subject": "vehicle",
                "deontic": "must",
                "verb": "stop",
                "complement": "behind the line at a junction",
            }
            "conditions":[
                {
                    "subject": "junction",
                    "verb": "has",
                    "complement": "a 'Stop' sign",
                },
                {
                    "subject": "junction",
                    "verb": "has",
                    "complement": "a solid white line across the road",
                },
            ]
        }
    },
    {
        "source_text": "Wait for a safe gap in the traffic before you move off.",
        "syntactic_structure": {
            "conclusion": {
                "subject": "vehicle",
                "deontic": "None",
                "verb": "move off",
                "complement": "",
            }
            "conditions":[
                {
                    "subject": "a safe gap",
                    "verb": "is",
                    "complement": "in the traffic",
                }
            ]
        }
    }
]
```

## Final remarks

- Extract the sentences and re-write them into the **singular tense**
- Replace subject "you" with "vehicle"
- Extract the JSON structure specificed.


""".strip()

PROMPT_GENERATOR = """
# Prompt: Transform Legal Text and JSON structure into Logical English (LE)

## Objective

Convert the provided legal text considering the JSON structure into Logical English (LE) following the step by step below.


- For each JSON object with conclusion and conditions, *identify the templates*. And explain using comment ("%") why you chose the templates.


**Conclusion**

- You should explicitly choose *one* of the provided templates to write the conclusion.

- Only create a new template, if and only if, the existing templates *do not fit*.
- Templates must be concise and generic enough.
- Each template has a syntax and an intent. You must check syntax and intent before choosing a template.
- Present the conclusion together with the corresponding template.

**Conditions**

Follow bellow some details on how to extract the conditions.

- Find the subject, the verb, and the specification.
- Use the verb as the predicate name.
- The arguments are the subject and specifications.
- You should explicitly choose *one* of the provided templates to write the conclusion.

```le
the templates are

*a thing* has *a thing*
*a thing* is *a thing*
*a thing* exists in *a thing*
```

- Only create a new template, if and only if, the existing templates *do not fit*.
- For each condition, present it together with the corresponding template.

5. *Build the rules* using the identified templates.

- Connect the rules and conditions using the templates and logical connections from Logical English.
- If the subject is "you", replace the agent in the templates with the constant *vehicle*.
- The ouput will be a single `le` code space delimited by triple ticks.

# Few shot examples

## Sentence to analyse

"**Box junctions.** These have criss-cross yellow lines painted on the road (see ‘[Road markings](/guidance/the-highway-code/road-markings)’). You **MUST NOT** enter the box until your exit road or lane is clear. However, you may enter the box and wait when you want to turn right, and are only stopped from doing so by oncoming traffic, or by other vehicles waiting to turn right. At signalled roundabouts you **MUST NOT** enter the box unless you can cross over it completely without stopping.

## JSON Object

```json
[
  {
    "source_text": "You MUST NOT enter the box until your exit road or lane is clear.",
    "syntactic_structure": {
      "conclusion": {
        "subject": "vehicle",
        "deontic": "must not",
        "verb": "enter",
        "complement": "the box"
      },
      "conditions": [
        {
          "subject": "exit road or lane",
          "verb": "is",
          "complement": "clear"
        }
      ]
    }
  },
  {
    "source_text": "However, you may enter the box and wait when you want to turn right, and are only stopped from doing so by oncoming traffic, or by other vehicles waiting to turn right.",
    "syntactic_structure": {
      "conclusion": {
        "subject": "vehicle",
        "deontic": "may",
        "verb": "enter and wait",
        "complement": "in the box"
      },
      "conditions": [
        {
          "subject": "vehicle",
          "verb": "wants",
          "complement": "to turn right"
        },
        {
          "subject": "vehicle",
          "verb": "is stopped",
          "complement": "by oncoming traffic"
        },
        {
          "subject": "vehicle",
          "verb": "is stopped",
          "complement": "by other vehicles waiting to turn right"
        }
      ]
    }
  },
  {
    "source_text": "At signalled roundabouts you MUST NOT enter the box unless you can cross over it completely without stopping.",
    "syntactic_structure": {
      "conclusion": {
        "subject": "vehicle",
        "deontic": "must not",
        "verb": "enter",
        "complement": "the box"
      },
      "conditions": [
        {
          "subject": "vehicle",
          "verb": "can cross",
          "complement": "over the box completely without stopping"
        }
      ]
    }
  }
]
```




## Previous templates

```le
the templates are:
*an agent* should *an action*.
*an agent* must *an action*.
*an agent* can *an action*.
*an agent* can *an action* at *a location*.
*an agent* must *an action* to *an agent*.
*an agent* must not *an action*.
*an agent* must not *an action* to *an agent*.
*an agent* must not *an action* at *a location*.
*an agent* should *an action* to *an agent*.
*an agent* should not *an action*.
*an agent* should not *an action* to *an agent*.

*an agent* must *an action* at *a location*.
*an agent* must not *an action*.
*an agent* must not *an action* at *a location*.

*an agent* sees *an item*.
*an agent* is at *a place*.
*an agent* is in *a place*.
*an agent* _is *a place*.
*an agent* cannot *an action*.
*a thing* has *a thing*
*a thing* is *a thing*
*a thing* exists in *a thing*
```

## Expected Output

### Step 1: Identify the templates for each JSON object

**JSON Object 1:** You MUST NOT enter the box until your exit road or lane is clear.

**Identifing templates**

**Conclusion:** *an agent* must not *an action*.
- Justification: it matches the deontic "must not" and the action "enter" and complement.

**Condition 1:** *a thing* is *a thing*.
- Justification: it matches the subject "exit road or lane" with thing and the complement "clear" with other thing.

**JSON Object 2:** However, you may enter the box and wait when you want to turn right, and are only stopped from doing so by oncoming traffic, or by other vehicles waiting to turn right.

**Identifing templates**

**Conclusion:** *an agent* can *an action*.
- Justification: it matches the deontic "may" and the action "enter and wait".

**Condition 1:** *an agent* intends to *an action*.
- Justification: it matches the intentions of doing an action which is turn right.

**Condition 2:** *an agent* is stopped by *an agent*. (New template)
- Justification: the subject "vehicle" and the complement "by oncoming traffic" or "by other vehicles waiting to turn right".

**JSON Object 3:** At signalled roundabouts you MUST NOT enter the box unless you can cross over it completely without stopping.

**Conclusion:** *an agent* must not *an action*.
- Justification: matches the deontic "must not" and the action "enter".

**Condition 1:** *an agent* cannot *an action*.
- Justification: it matches the subject "vehicle" and the complement "cross over the box completely without stopping".

## Step 2: Present the final LE code using the templates and rules

```le
the templates are:

% -------------
% Existing templates used
*an agent* must not *an action*.
*a thing* is *a thing*.
*an agent* can *an action*.
*an agent* intends to *an action*.
*an agent* cannot *an action*.

% -------------
% New templates
*an agent* is stopped by *an agent*.

% -------------
% New rules

% JSON Object 1
a vehicle must not enter the box
if exit road or lane is not clear.

% JSON Object 2
vehicle can enter the box and wait
if vehicle intends to turn right
and vehicle is stopped by oncoming traffic
    or vehicle is stopped by other vehicles waiting to turn right.

% JSON Object 3
vehicle must not enter the box
if vehicle cannot cross over the box completely without stopping.
```


## Logical English Instructions:

In the following section, we will explain the details on Logical English syntax.

#### Basic Form

- **Structure:** `conclusion if conditions`
- **Example:** `a person P is a British citizen if P is born in the United Kingdom after commencement and ...`

#### Templates

- **Purpose:** Declare a fixed predicate with variable arguments.
- **Structure:** Use simple or compound verbs and noun phrases.
- **Example:** `a person P is a British citizen`
- **Conciseness:** Templates must be concise. If a phrase is complex, it should be broken down into many templates.

#### Variables

- **Introduction:** Use indefinite articles (a/an) to introduce variables.
- **Example:** `a person P`
- **Reference:** Use definite articles (the) to refer back to variables.
- **Example:** `the person P`

#### Logical Constructs

- **AND:** Used to combine multiple conditions that must all be true.
    **Example:** `condition1 and condition2`
- **OR:** Used to combine conditions where at least one must be true.
    **Example:** `condition1 or condition2`
- **It is not the case that:** Used to negate a condition.
    **Example:** `it is not the case that condition`

---

### Example Transformations

#### British Nationality Act 1981

**Natural Language:**
"A person born in the UK after commencement shall be a British citizen if at the time of birth his father or mother is a British citizen or settled in the UK."

**LE Form:**
```le
the templates are:
*a person* acquires british citizenship on *a date*,
*a person* is born in the uk on *a date*,
*a date* is after commencement,
*a person* is the mother of *a person*,
*a person* is the father of *a person*,
*a person* is a british citizen on *a date*,
*a person* is settled in the uk on *a date*.

the knowledge base citizenship includes:
a person acquires british citizenship on a date
if the person is born in the uk on the date
and the date is after commencement
and an other person is the mother of the person
    or the other person is the father of the person
and the other person is a british citizen on the date
    or the other person is settled in the uk on the date.
```

#### ISDA Master Agreement
**Natural Language:**
"If at any time an Event of Default with respect to a party (the 'Defaulting Party') has occurred and is then continuing, the other party (the 'Non-defaulting Party') may, by not more than 20 days notice to the Defaulting Party specifying the relevant Event of Default, designate a day not earlier than the day such notice is effective as an Early Termination Date in respect of all outstanding Transactions."

**LE Form:**
```le
the templates are:

it is permitted that *an eventuality*,
*a party* designates that *an eventuality*,
*an event* occurs at *a time*.
*an event* of Default occurs with respect to *a party* at *a time*,
*an event* is continuing at *a time*,
*a party* gives notice to *a party* at *a time* that *a message*,
*a date* is on or before *another date*,
*a time* and *a time* are at most *a number* days apart, the Schedule specifies that *a specification*,
Automatic Early Termination applies to *a party* for *an event* of Default.

the knowledge base eventuality includes:

it is permitted that a party designates
that Early Termination in respect of all outstanding Transactions occurs at a time T3

if an Event of Default occurs with respect to another party at a time T1
and the Event is continuing at a time T2
and the party gives notice to the other party at T2
that the Event occurs at T1
and T2 is on or before T3
and T3 and T2 are at most 20 days apart
and it is not the case that
the Schedule specifies that Automatic Early Termination applies to the other party for the Event of Default.
```

#### Loan Agreement
**Natural Language:**
"5. The Borrower will be in default under this agreement upon the occurrence of any of the
following events or conditions, provided they shall remain uncured within a period of two days
after notice is given to Borrower by Lender of their occurrence (such an uncured event an “Event
of Default”):
(a) Borrower shall fail to make timely payment of any amount due to Lender hereunder;
(b) Any of the representation or warranties of Borrower under this agreement shall prove
untrue;
(c) Borrower shall fail to perform any of its covenants under this agreement;
(d) Borrower shall file for bankruptcy or insolvency under any applicable federal or state law.
A default will be cured by the Borrower (i) remedying the potential event of default and (ii)
giving effective notice of such remedy to the Lender. In the event of multiple events of default,
the first to occur shall take precedence for the purposes of specifying outcomes under this
agreement.
6. Acceleration on Default
Upon the occurrence of an Event of Default all outstanding payments under this agreement will
become immediately due and payable, including both principal and interest amounts, without
further notice, presentment, or demand to the Borrower."

**LE Form:**
```le
the borrower fails on a date to fulfil an obligation if
the obligation is
that the borrower pays an amount to the lender on the date
and it is not the case that
the borrower pays the amount to the lender on the date.

the borrower defaults on a date D3 if
the borrower must fulfil an obligation
and the borrower fails on a date D0 to fulfil the obligation
and the lender gives notice to the borrower on a date D1
that the borrower fails on D0 to fulfil the obligation
and D3 is 2 days after D1
and it is not the case that
the borrower cures on a date D2 the failure of the obligation
and D2 is on or before D3.

the borrower cures on a date D the failure of an obligation
if the obligation is
that the borrower pays an amount to the lender on a date DO
and the borrower pays the amount to the lender
on a date D1
and the borrower gives notice to the lender on a date D2
that the borrower pays the amount to the lender on D1
and D is the latest of D1 and D2.
```

## Final Remarks

Your task is to transform JSON and source text into Logical English.
To do so, follow the step by step presented before. To write Logical English follow the cheatsheet
Fulfill the following steps:

1. For each JSON object with conclusion and conditions, identify the templates.
2. Present the final LE code using the templates and rules.
    - Split new templates from old templates using a comment "% New templates"
    - Split new rules from old rules using a comment "% New rules"

""".strip()

In [2]:
user_prompt_json = """
## Sentence to analyse

"@SOURCE_TEXT"

""".strip()

user_prompt_generator = """
## Sentence to analyse

"@SOURCE_TEXT"

## JSON Object

```json
@JSON_OBJECT
```

## Previous templates

```le
@PREVIOUS_TEMPLATES
```

""".strip()

In [3]:
from pipeline.rule_generation import process_existing_le
import re
import logging
from dotenv import load_dotenv

rules_to_ignore = [
]
load_dotenv()

markdown_dataset = open("../Using the road (159 to 203).md").read().strip()

existing_logical_english = open("../highway_code.le").read().strip()
existing_templates, existing_kb = process_existing_le(existing_logical_english)

existing_le = "\n\n".join([existing_templates, existing_kb])
logging.info(f"-----\n{existing_templates}\n=====\n{existing_kb}")



2025-07-12 13:41:14,361 - INFO - Logging setup complete.
2025-07-12 13:41:14,364 - INFO - -----
the templates are:
*an agent* should *an action*.
*an agent* must *an action*.
*an agent* can *an action*.
*an agent* can *an action* at *a location*.
*an agent* must *an action* to *an agent*.
*an agent* must not *an action*.
*an agent* must not *an action* to *an agent*.
*an agent* must not *an action* at *a location*.
*an agent* should *an action* to *an agent*.
*an agent* should not *an action*.
*an agent* should not *an action* to *an agent*.

*an agent* must *an action* at *a location*.
*an agent* must not *an action*.
*an agent* must not *an action* at *a location*.

*an agent* intends to *an action*.

*an agent* sees *an item*.
*an agent* is at *a place*.
*an agent* is in *a place*.
*an agent* _is *a place*.
*an agent* cannot *an action*.
*a thing* has *a thing*
*a thing* is *a thing*
*a thing* exists in *a thing*
=====
the knowledge base highway_code includes:

a vehicle must an act

In [4]:
from pipeline.rule_generation import parse_markdown_rules

parsed_source_text = parse_markdown_rules(markdown_dataset)
junction_rules = parsed_source_text["Road junctions (rules 170 to 183)"]

In [5]:



from together import Together


def api_call(system_prompt=None, user_prompt="", options=None):
    if options is None:
        options = {}

    # Initialize client
    client = Together()

    # Build message list
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": user_prompt})

    try:
        # Send API call with dynamic options
        response = client.chat.completions.create(
            model=options.get("model", "meta-llama/Llama-3.3-70B-Instruct-Turbo-Free"),
            messages=messages,
            **{k: v for k, v in options.items() if k != "model"}  # Pass all other options
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"


import json
import re


def extract_json_from_markdown(markdown_text: str) -> dict | None:
    """
    Parses a JSON object from a Markdown code block.

    Args:
        markdown_text: A string containing the Markdown output from an LLM.

    Returns:
        A dictionary parsed from the first JSON code block found,
        or None if no valid JSON block is found.
    """
    # This regex pattern finds a JSON code block and captures its content.
    # It handles potential leading/trailing whitespaces and newlines.
    pattern = r"```json\s*\n(.*?)\n\s*```"

    # re.DOTALL makes '.' match any character, including newlines.
    match = re.search(pattern, markdown_text, re.DOTALL)

    if match:
        json_string = match.group(1).strip()
        try:
            return json.loads(json_string)
        except json.JSONDecodeError as e:
            logging.error(f"Error decoding JSON: {e}")
            return None

    return None


from typing import List, Dict, Any


def filter_content(data: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Filters a list of objects and cleans their 'conditions' list.

    An object is kept if its 'conclusion' is valid. Within each kept object,
    any 'condition' that does not have both a subject and a verb is removed.

    Args:
        data: A list of dictionary objects to be filtered and cleaned.

    Returns:
        A new list containing the valid objects with cleaned condition lists.
    """
    filtered_list = []

    if not data:
        return None

    for item in data:
        # Use deepcopy to avoid modifying the original input list
        item_copy = json.loads(json.dumps(item))
        structure = item_copy.get("syntactic_structure")

        if not structure:
            continue

        # 1. Validate the 'conclusion'
        conclusion = structure.get("conclusion")
        if not conclusion:
            continue

        is_conclusion_valid = isinstance(conclusion, dict) and all([
            conclusion.get("subject") == "vehicle",
            conclusion.get("deontic"),
            conclusion.get("verb"),
            conclusion.get("verb") != "watch out"
        ])

        if not is_conclusion_valid:
            continue

        # 2. Clean the 'conditions' list
        original_conditions = structure.get("conditions", [])
        if original_conditions:
            valid_conditions = []
            for condition in original_conditions:
                # A condition is valid only if both 'subject' and 'verb' are present and not empty
                if isinstance(condition, dict) and condition.get("subject") and condition.get("verb"):
                    valid_conditions.append(condition)

            # Replace the old conditions list with the cleaned one
            structure["conditions"] = valid_conditions

        # 3. If the conclusion was valid, add the cleaned item to the result
        filtered_list.append(item_copy)

    return filtered_list


def extract_json_le(source_text, previous_le):
    global user_prompt_generator, user_prompt_json, PROMPT_JSON_EXTRACTION

    filled_user_prompt_json = str(user_prompt_json)
    filled_user_prompt_json = filled_user_prompt_json.replace("@SOURCE_TEXT", source_text)
    # filled_user_prompt = filled_user_prompt.replace("@PREVIOUS_TEMPLATES", previous_le)

    logging.info("===============================================================")
    logging.info("-----------------   EXTRACTING JSON - SYSTEM PROMPT  ----------")
    logging.info("---------------------------------------------------------------")
    logging.info(f"\n{PROMPT_JSON_EXTRACTION}")
    logging.info("===============================================================")


    logging.info("===============================================================")
    logging.info("-----------------   EXTRACTING JSON - USER PROMPT  ------------")
    logging.info("---------------------------------------------------------------")
    logging.info(f"\n{filled_user_prompt_json}")
    logging.info("===============================================================")

    json_structure = None
    while not json_structure:
        response = api_call(
            system_prompt=PROMPT_JSON_EXTRACTION,
            user_prompt=filled_user_prompt_json,
            options={
                "model": "meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
                "temperature": 0.6,
                "max_tokens": 6000
            }
        )
        # logging.info(f"JSON extraction Response: \n\n====\n{response}\n=======")

        json_structure = extract_json_from_markdown(response)
        json_structure = filter_content(json_structure)


    logging.info("===============================================================")
    logging.info("--------------   EXTRACTING JSON - VALIDATED OUTPUT  ----------")
    logging.info("---------------------------------------------------------------")
    logging.info(f"\n{json.dumps(json_structure, indent=1)}")
    logging.info("===============================================================")

    return json_structure


def extract_le_from_json(source_text: dict, previous_le: dict, json_structure: dict):
    global user_prompt_generator, PROMPT_GENERATOR

    # Fill the user prompt with given information
    current_user_prompt_generator = str(user_prompt_generator)
    current_user_prompt_generator = current_user_prompt_generator.replace("@SOURCE_TEXT", source_text)
    current_user_prompt_generator = current_user_prompt_generator.replace("@JSON_OBJECT",
                                                                          json.dumps(json_structure, indent=2))
    current_user_prompt_generator = current_user_prompt_generator.replace("@PREVIOUS_TEMPLATES", previous_le)

    logging.info("===============================================================")
    logging.info("-----------------   CONVERT LE - SYSTEM PROMPT  ---------------")
    logging.info("---------------------------------------------------------------")
    logging.info(f"\n{PROMPT_GENERATOR}")
    logging.info("===============================================================")


    logging.info("===============================================================")
    logging.info("-----------------   CONVERT LE - USER PROMPT  -----------------")
    logging.info("---------------------------------------------------------------")
    logging.info(f"\n{current_user_prompt_generator}")
    logging.info("===============================================================")

    # Send the prompt to the API
    le_output = None

    while not le_output:
        response = api_call(
            system_prompt=PROMPT_GENERATOR,
            user_prompt=current_user_prompt_generator,
            options={
                "model": "meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
                "temperature": 0.6,
                "max_tokens": 2000
            }
        )

        # TODO: implement validation
        le_output = response

    # Extract templates and rules
    logging.info("===============================================================")
    logging.info("-------------------   CONVERT LE - OUTPUT  --------------------")
    logging.info("---------------------------------------------------------------")
    logging.info(f"\n{le_output}")
    logging.info("===============================================================")

    # Return the string with templates and rules.
    pass




In [6]:
for rule, source_text in junction_rules.items():
    if len(source_text.strip()) == 0 or rule.strip() in rules_to_ignore:
        continue

    logging.info("\n\n\n\n\n\n\n")
    logging.info("=" * 50)
    logging.info(f"Processing: {rule}")
    logging.info(f"Source text: \n\"{source_text}\"")
    logging.info("=" * 50)

    # Extract JSON
    json_content = extract_json_le(
        source_text=source_text,
        previous_le=existing_templates
    )

    # Extract LE
    # ...
    extract_le_from_json(
        source_text=source_text,
        previous_le=existing_templates,
        json_structure=json_content
    )

    # Extract Scenarios from templates
    # ...

    # Syntax checking
    #

2025-07-12 13:41:14,485 - INFO - 







2025-07-12 13:41:14,486 - INFO - ==================================================
2025-07-12 13:41:14,486 - INFO - Processing: Rule 170
2025-07-12 13:41:14,487 - INFO - Source text: 
"Take extra care at junctions. You should

*   watch out for cyclists, motorcyclists and pedestrians including powered wheelchairs/mobility scooter users as they are not always easy to see. Be aware that they may not have seen or heard you if you are approaching from behind
*   give way to pedestrians crossing or waiting to cross a road into which or from which you are turning. If they have started to cross they have priority, so give way (see [Rule H2](/guidance/the-highway-code/introduction#ruleh2))
*   remain behind cyclists, horse riders, horse drawn vehicles and motorcyclists at junctions even if they are waiting to turn and are positioned close to the kerb
*   watch out for long vehicles which may be turning at a junction ahead; they may have to use the whol